# Images: a use of NumPy arrays

Let's load up some useful modules:

In [ ]:
import numpy as np

# Let's import OpenCV
# pip install opencv-python if needed.
import cv2 as cv

# We'll use the matplotlib library
import matplotlib.pyplot as plt

# Make the figures a bit bigger than the default:
plt.rcParams['figure.figsize'] = [10,8]

Let's have a look for images:

In [ ]:
!ls *.{jpg,png}

In [ ]:
img_bgr = cv.imread("craignish.jpg")  # Use the non-rotated version!

img = cv.cvtColor(img_bgr, cv.COLOR_BGR2RGB)

plt.imshow(img)
plt.title("Craignish, Scotland");

Note the semicolon.

In [ ]:
img.shape

Remember that the first axis in NumPy normally identifies the *row*.

In [ ]:
corner = img[ -200:, :300]
plt.imshow(corner);

---

## A behind-the-scenes look at NumPy arrays

Let's look under the hood at the original image:

In [ ]:
img.dtype

In [ ]:
img.itemsize

In [ ]:
img.shape

_Round here, he's known as Strider...._

In [ ]:
img.strides

And now at our small corner:

In [ ]:
corner.shape

In [ ]:
corner.strides

That's interesting!  Why is the stride still the same?

In [ ]:
corner.flags.owndata

'corner' is simply a *view* onto the same 'img' data, but with different offset and shape.

### Another example: transposing

Arrays have a property T, which represents their transpose:

In [ ]:
reds = img[:,:,0]

print(reds.shape)
print(reds.T.shape)

In [ ]:
reds.dtype

In [ ]:
plt.imshow(reds, cmap="gray");

In [ ]:
plt.imshow(reds.T, cmap="gray");

We can compare:

In [ ]:
reds.strides

In [ ]:
reds.T.strides

Similarly, we can skip rows or columns just by changing the stride:

In [ ]:
squished = reds[:,::3]
plt.imshow(squished, cmap="gray");

In [ ]:
squished.strides

NumPy is generally very good at using views and not copying data unless it needs to.  You do need to remember this when assigning to arrays, though:

In [ ]:
corner[:20,:20] = [255, 255,0]
plt.imshow(img)   # Note we're drawing 'img'

There's are various copy() methods and functions if needed, and you may sometimes want to arrange your data order to make optimal use of caches.

In [ ]:
gridded = img.copy()
gridded[:, ::30] = [255,0,0]
gridded[::30, :] = [255,0,0]
plt.imshow(gridded);

**Question**: Some lines of the grid are missing?  Why?

---

### An esoteric aside

In [ ]:
a = np.arange(10)
a

In [ ]:
a.dtype

In [ ]:
a.strides

Get a custom view onto the same data:

In [ ]:
np.lib.stride_tricks.as_strided(a, shape=(7,4), strides=(8,8))

OK, back to the pictures...

---

### Histogram equalisation and some simple matplotlib

Let's look at greyscale of the whole image:

In [ ]:
grey_img = cv.cvtColor(img, cv.COLOR_RGB2GRAY)

plt.imshow(grey_img, cmap="gray");


In [ ]:
hist, bins = np.histogram(grey_img, bins=256, range=[0,256])
hist, bins

Get the cumulative sum of the histogram values:

In [ ]:
cdf = hist.cumsum()
cdf

Let's do a simple plot:

In [ ]:
plt.plot(bins[:-1], hist);

In [ ]:
plt.axes?

A slightly more complex plot, showing the histogram and this cumulative distribution function:

In [ ]:
fig, ax1 = plt.subplots(figsize=(12,6))

# Plot the histogram using the normal y axis
ax1.bar(bins[1:], hist, 1, color='r')
ax1.tick_params(axis='y', labelcolor='r')

# Make a second y axis that shares the x axis
ax2 = ax1.twinx()

# And on that we'll plot the CDF
ax2.plot(cdf, color='b')
ax2.tick_params(axis='y', labelcolor='b')

# Some formatting stuff
fig.tight_layout()
fig.legend(('histogram', 'cdf'), loc = 'upper left');

# plt.savefig("histo.pdf")
plt.show()

Use OpenCV's standard histogram-based equalisation:

In [ ]:
equ_img = cv.equalizeHist(grey_img)

And compare the results:

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(14,6))
axs[0].imshow(grey_img, cmap="gray")
axs[0].set_title("original")

axs[1].imshow(equ_img, cmap="gray")
axs[1].set_title("equalised")

plt.show()

What does the histogram look like for this new image?

In [ ]:
hist,bins = np.histogram(equ_img.flatten(),256,[0,256])
cdf = hist.cumsum()

fig, ax1 = plt.subplots(figsize=(12,6))

# Plot the histogram using the normal y axis
ax1.bar(bins[1:], hist, 1, color='r')
ax1.tick_params(axis='y', labelcolor='r')

# Make a second y axis that shares the x axis
ax2 = ax1.twinx()
ax2.tick_params(axis='y', labelcolor='b')
# And on that we'll plot the CDF
cdf = hist.cumsum()
ax2.plot(cdf, color = 'b')

# Some formatting stuff
fig.tight_layout()
fig.legend(('cdf','histogram'), loc = 'upper left');

plt.show()


Too gritty?  Perhaps we can blend the two?

In [ ]:
for i in range(0,5):
    mix = i / 4.0
    blend = (equ_img * mix) + (grey_img * (1.0-mix))
    print("original:", 1-mix, "normalised:", mix)
    plt.imshow(blend, cmap="gray")
    plt.show()

# Adding some colour overlays.

OK, let's use the original grey image and make it RGB again so that we can then add other colours.  

There are various ways you could do this.

In [ ]:
grey_img.shape

In [ ]:
h, w = grey_img.shape

mono_img = np.zeros( (h, w, 3), dtype=np.uint8)
mono_img.shape

In [ ]:
# An ellipsis ... means 'as many dimensions as you need'
mono_img[...,0] = grey_img
mono_img[...,1] = grey_img
mono_img[...,2] = grey_img
mono_img

In [ ]:
plt.imshow(mono_img)  # No cmap needed now
plt.show()

This works, but a faster way is using broadcasting explicity.

We can turn a 2D array into a 3D array by adding a new axis:

In [ ]:
grey_img_3d = grey_img[..., np.newaxis]  # np.newaxis or 'None'
grey_img_3d.shape

That final dimension is of size '1', so we can use it for broadcasting:

In [ ]:
mono_img = np.broadcast_to(  grey_img[..., np.newaxis], (h, w, 3) )
mono_img

In [ ]:
plt.imshow(mono_img)
plt.show()

This is a *colour* image, it's just that all the colour components have the same value.  We can change that:

In [ ]:
lowlights = np.where( mono_img > 40, img, [255,0,255] )

plt.imshow(lowlights);

#### Go light blues!

In [ ]:
blues = img[...,2]
brightbluemask = (blues > 200)
brightbluemask

In [ ]:
plt.imshow(brightbluemask, cmap="gray");

We can use Numpy's indexing capabilities:

In [ ]:
lowlights[ brightbluemask ] = [ 0, 0, 255 ]
plt.imshow(lowlights);

## Matplotlib alternatives

Matplotlib can do a lot -- see [examples on matplotlib.org](https://matplotlib.org/gallery/index.html) -- and has many advantages, but there are several alternatives.

* Bokeh
* Plot.ly
* ggplot variants (for example plotnine)